In [1]:
import numpy as np

In [23]:
def FDR_above_threshold_v1(orig, null, FDR):
    """
    :param orig: vector of beta values from true distribution
    :param null: vector of beta values from null distribution
    :param FDR: False Discovery Rate
    :return: threshold t such that #(null > t)/ [#(null > t) + #(orig > t)]
    """
    pos_values = np.sort(orig[np.where(orig > 0)])

    for pos_value in pos_values:
        origP = get_num_above(orig, pos_value)
        nullP = get_num_above(null, pos_value)
        fdr = nullP * 1.0 / (nullP + origP)
        if fdr < FDR:
            return pos_value

    return None

def get_num_above(betas, threshold):
    """
    :param betas: vector of beta values
    :param threshold:
    :return: # betas >= threshold
    """
    return len(np.where(betas >= threshold)[0])

def FDR_above_threshold_v2(orig, null, FDR):
    """
    Does from the minimum up.
    :param orig: vector of beta values from true distribution
    :param null: vector of beta values from null distribution
    :param FDR: False Discovery Rate
    :return: smallest threshold t such that #(null >= t)/ [#(null >= t) + #(orig >= t)]
    """
    orig_values = np.sort(orig)
    null_values = np.sort(null)
    
    
    orig_ind = 0
    orig_above = len(orig_values)
    orig_max = len(orig_values)
    
    null_ind = 0
    null_above = len(null_values)
    null_max = len(null_values)
    
    while (orig_ind < orig_max and null_ind < null_max):
        orig_val = orig_values[orig_ind]
        null_val = null_values[null_ind]
        
        if orig_val > null_val:
            orig_ind += 1
            orig_above -= 1
            thresh = orig_val
        elif orig_val < null_val:
            null_ind += 1
            null_above -= 1
            thresh = null_val
        else:
            orig_ind += 1
            orig_above -= 1
            null_ind += 1
            null_above -= 1
            
            thresh = orig_val
        
        fdr = (null_above * 1.0)/(orig_above + null_above)
        
        if fdr < FDR:
            return thresh
    
    return None
            
    
    
def FDR_above_threshold_v3(orig, null, FDR):
    """
    :param orig: vector of beta values from true distribution
    :param null: vector of beta values from null distribution
    :param FDR: False Discovery Rate
    :return: smallest threshold t such that #(null >= t)/ [#(null >= t) + #(orig >= t)] < FDR
    """
    orig_values = np.sort(orig)
    null_values = np.sort(null)
    
    
    orig_ind = len(orig_values) -1
    orig_above = 0
    
    null_ind = len(null_values) - 1
    null_above = 0
    
    
    thresh = None
    
    while (orig_ind >= 0 and null_ind >= 0):
        orig_val = orig_values[orig_ind]
        null_val = null_values[null_ind]
        
        prev_thresh = thresh
        
        if orig_val > null_val:
            orig_ind -= 1
            orig_above += 1
            thresh = orig_val
            
        elif orig_val < null_val:
            null_ind -= 1
            null_above += 1
            thresh = null_val
        else:
            orig_ind -= 1
            orig_above += 1
            null_ind -= 1
            null_above += 1
            
            thresh = orig_val
        
        
        try:
            fdr = (null_above * 1.0)/(orig_above + null_above)
        
        # case where the highest value is the null.
        except ZeroDivisionError:
            return None
        
        # return the previous threshold if this one fails
        if fdr >= FDR:
            return prev_thresh
    
    # Case 1: we ran out of originals. Then just return the current
    if orig_ind < 0:
        return thresh
    
    # Case 2: we ran out of nulls, then we can safely take the rest of the originals
    elif null_ind < 0:
        return orig_values[0]
    else:
        raise ValueError("Should never get to this code.")

In [24]:

np.random.seed(123)

size = 50
FDR = 0.1

null = np.absolute(np.random.normal(5, 1, size))
orig = np.absolute(np.random.normal(5, 3, size))

print "Null", np.sort(null)
print "Orig", np.sort(orig)

%time t1 = FDR_above_threshold_v1(orig, null, FDR)
%time t2 = FDR_above_threshold_v2(orig, null, FDR)
%time t3 = FDR_above_threshold_v3(orig, null, FDR)

print "v1 thresh", t1
print "v2 thresh", t2
print "v3 thresh", t3

Null [ 2.20141089  2.57332076  3.2284669   3.27233051  3.49370529  3.5713193
  3.74611933  3.9143694   4.06416613  4.12046366  4.1332596   4.1382451
  4.19463348  4.30012277  4.32111385  4.361098    4.3622485   4.42139975
  4.55601804  4.56564872  4.57108737  4.60910021  4.74438063  4.82636432
  4.85993128  4.90529103  4.98816951  5.00284592  5.2829785   5.28362732
  5.33858905  5.3861864   5.41291216  5.57380586  5.68822271  5.73736858
  5.9071052   5.92746243  5.97873601  5.99734545  6.0040539   6.17582904
  6.26593626  6.49073203  6.49138963  6.65143654  7.18678609  7.20593008
  7.23814334  7.39236527]
Orig [  0.18211172   0.90958537   1.02120362   1.11774403   1.19794385
   1.36243061   1.36930105   1.40509657   1.70839086   1.7422928
   1.74629626   1.88363537   2.48744983   2.50653505   2.60581179
   2.68187386   2.80261404   2.93339305   3.78990189   3.8624707
   3.99496771   4.05572556   4.30072382   4.62191124   5.08904969
   5.13647024   5.49332369   5.54310539   5.59857222  

In [25]:
# edge cases: orig all above null. Should return least of orig.

FDR = 0.1

null = np.arange(0, 5, 1.0)
orig = np.arange(6, 20, 1.0)

print "Null", np.sort(null)
print "Orig", np.sort(orig)


%time t1 = FDR_above_threshold_v1(orig, null, FDR)
%time t2 = FDR_above_threshold_v2(orig, null, FDR)
%time t3 = FDR_above_threshold_v3(orig, null, FDR)

print "v1 thresh", t1
print "v2 thresh", t2
print "v3 thresh", t3



Null [ 0.  1.  2.  3.  4.]
Orig [  6.   7.   8.   9.  10.  11.  12.  13.  14.  15.  16.  17.  18.  19.]
CPU times: user 103 µs, sys: 22 µs, total: 125 µs
Wall time: 103 µs
CPU times: user 83 µs, sys: 140 µs, total: 223 µs
Wall time: 217 µs
CPU times: user 36 µs, sys: 0 ns, total: 36 µs
Wall time: 40.1 µs
v1 thresh 6.0
v2 thresh None
v3 thresh 6.0


In [27]:
# edge cases: orig barely above null. When first null arrivees, goes above the fdr
# then should choose the previous threshold

FDR = 0.05

null = np.arange(0, 8, 1.0)
orig = np.arange(5, 20, 1.0)

print "Null", np.sort(null)
print "Orig", np.sort(orig)


%time t1 = FDR_above_threshold_v1(orig, null, FDR)
%time t2 = FDR_above_threshold_v2(orig, null, FDR)
%time t3 = FDR_above_threshold_v3(orig, null, FDR)

print "v1 thresh", t1
print "v2 thresh", t2
print "v3 thresh", t3



Null [ 0.  1.  2.  3.  4.  5.  6.  7.]
Orig [  5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.  16.  17.  18.  19.]
CPU times: user 86 µs, sys: 29 µs, total: 115 µs
Wall time: 95.8 µs
CPU times: user 219 µs, sys: 100 µs, total: 319 µs
Wall time: 253 µs
CPU times: user 44 µs, sys: 0 ns, total: 44 µs
Wall time: 48.2 µs
v1 thresh 8.0
v2 thresh None
v3 thresh 8.0


In [28]:
# edge cases: orig barely above null. When first null arrivees, goes above the fdr
# then should choose the previous threshold

FDR = 0.05

null = np.arange(0, 21, 1.0)
orig = np.arange(5, 20, 1.0)

print "Null", np.sort(null)
print "Orig", np.sort(orig)


%time t1 = FDR_above_threshold_v1(orig, null, FDR)
%time t2 = FDR_above_threshold_v2(orig, null, FDR)
%time t3 = FDR_above_threshold_v3(orig, null, FDR)

print "v1 thresh", t1
print "v2 thresh", t2
print "v3 thresh", t3

Null [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.]
Orig [  5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.  16.  17.  18.  19.]
CPU times: user 197 µs, sys: 155 µs, total: 352 µs
Wall time: 330 µs
CPU times: user 215 µs, sys: 221 µs, total: 436 µs
Wall time: 271 µs
CPU times: user 53 µs, sys: 65 µs, total: 118 µs
Wall time: 106 µs
v1 thresh None
v2 thresh None
v3 thresh None


In [30]:

np.random.seed(123)

size = 1000
FDR = 0.1

null = np.absolute(np.random.normal(5, 1, size))
orig = np.absolute(np.random.normal(5, 3, size))

print "Null", np.sort(null)
print "Orig", np.sort(orig)

%time t1 = FDR_above_threshold_v1(orig, null, FDR)
%time t2 = FDR_above_threshold_v2(orig, null, FDR)
%time t3 = FDR_above_threshold_v3(orig, null, FDR)

print "v1 thresh", t1
print "v2 thresh", t2
print "v3 thresh", t3

Null [ 1.76894499  1.83294467  2.20141089  2.2055277   2.21188712  2.36856196
  2.4429454   2.55692421  2.56153944  2.57332076  2.57516367  2.72738229
  2.74846501  2.76358265  2.76383116  2.81291407  2.83059204  2.84750656
  2.87689965  2.87770112  2.89507157  2.9476016   2.94978232  2.95790358
  2.97495424  2.99141453  3.02211207  3.05785046  3.06823102  3.07441311
  3.07628427  3.10482441  3.11150762  3.12313134  3.12960252  3.1361157
  3.13802429  3.15796816  3.16933364  3.19237872  3.20611108  3.21347315
  3.22475716  3.22622865  3.2284669   3.23710227  3.24459816  3.24892965
  3.26901055  3.27233051  3.29100822  3.29850391  3.30794758  3.33231077
  3.33952493  3.34630216  3.36060303  3.37924802  3.39403724  3.39459026
  3.3948192   3.3955636   3.39955947  3.41279357  3.42255121  3.42376715
  3.4294714   3.43074387  3.43155995  3.43896949  3.4694871   3.48189202
  3.49370529  3.4965416   3.51804154  3.52702239  3.53488378  3.54931686
  3.5713193   3.579787    3.58127409  3.5821055